In [1]:
import pandas as pd
 
import json

prefix = 's3://czbiohub-seqbot/fastqs/180628_A00111_0168_AHFVJVDMXX/rawdata/'

aws_s3_ls = ! aws s3 ls $prefix

In [2]:
print(aws_s3_ls)

['2018-06-30 17:10:48 17498060325 Undetermined_S0_R1_001.fastq.gz', '2018-06-30 17:10:48 17934291414 Undetermined_S0_R2_001.fastq.gz', '2018-06-30 17:10:48 135904490012 tick_1_S1_R1_001.fastq.gz', '2018-06-30 17:10:48 140224587456 tick_1_S1_R2_001.fastq.gz', '2018-06-30 17:10:48 153875834035 tick_2_S2_R1_001.fastq.gz', '2018-06-30 17:29:13 159886321067 tick_2_S2_R2_001.fastq.gz']


In [3]:
from io import StringIO

data = '\n'.join(aws_s3_ls)
aws_samples = pd.read_table(StringIO(data), sep='\s+', header=None)
aws_samples

,0,1,2,3
0,2018-06-30,17:10:48,17498060325,Undetermined_S0_R1_001.fastq.gz
1,2018-06-30,17:10:48,17934291414,Undetermined_S0_R2_001.fastq.gz
2,2018-06-30,17:10:48,135904490012,tick_1_S1_R1_001.fastq.gz
3,2018-06-30,17:10:48,140224587456,tick_1_S1_R2_001.fastq.gz
4,2018-06-30,17:10:48,153875834035,tick_2_S2_R1_001.fastq.gz
5,2018-06-30,17:29:13,159886321067,tick_2_S2_R2_001.fastq.gz


In [7]:
aws_samples['id'] = aws_samples[3].str.extract('(.+)_R[12]')
aws_samples['read_number'] = aws_samples[3].str.extract('_(R[12])_')
aws_samples['reads'] = prefix + aws_samples[3]
aws_samples['output'] = 's3://tick-genome/dna/2018-06-28/pre-assembly_quality_control/reports/'
aws_samples

,0,1,2,3,id,reads,output,read_number
0,2018-06-30,17:10:48,17498060325,Undetermined_S0_R1_001.fastq.gz,Undetermined_S0,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://tick-genome/dna/2018-06-28/pre-assembly_q...,R1
1,2018-06-30,17:10:48,17934291414,Undetermined_S0_R2_001.fastq.gz,Undetermined_S0,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://tick-genome/dna/2018-06-28/pre-assembly_q...,R2
2,2018-06-30,17:10:48,135904490012,tick_1_S1_R1_001.fastq.gz,tick_1_S1,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://tick-genome/dna/2018-06-28/pre-assembly_q...,R1
3,2018-06-30,17:10:48,140224587456,tick_1_S1_R2_001.fastq.gz,tick_1_S1,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://tick-genome/dna/2018-06-28/pre-assembly_q...,R2
4,2018-06-30,17:10:48,153875834035,tick_2_S2_R1_001.fastq.gz,tick_2_S2,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://tick-genome/dna/2018-06-28/pre-assembly_q...,R1
5,2018-06-30,17:29:13,159886321067,tick_2_S2_R2_001.fastq.gz,tick_2_S2,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://tick-genome/dna/2018-06-28/pre-assembly_q...,R2


In [10]:
samples = aws_samples.pivot(index='id', values='reads', columns='read_number')
samples['id'] = samples.index
samples = samples.rename(columns={'R1': "read1", "R2": 'read2'})
samples['libprep'] = "nextera"
samples['output'] = 's3://tick-genome/dna/2018-06-28/adapter_trimmed'
samples

read_number,read1,read2,id,libprep,output
id,,,,,
Undetermined_S0,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,Undetermined_S0,nextera,s3://tick-genome/dna/2018-06-28/adapter_trimmed
tick_1_S1,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,tick_1_S1,nextera,s3://tick-genome/dna/2018-06-28/adapter_trimmed
tick_2_S2,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,s3://czbiohub-seqbot/fastqs/180628_A00111_0168...,tick_2_S2,nextera,s3://tick-genome/dna/2018-06-28/adapter_trimmed


In [13]:
folder = '../pre_assembly_qc/adapter_removal/'
! mkdir $folder

In [14]:
cd $folder

/Users/olgabot/code/tick-genome/pre_assembly_qc/adapter_removal


In [15]:
samples.to_csv(f'samples.csv', index=True)

In [19]:
config = dict(program='../../reflow/adapter_removal.rf', runs_file='samples.csv')

with open('config.json', 'w') as f:
    json.dump(config, f)

In [20]:
ls -lha

total 16
drwxr-xr-x  4 olgabot  staff   136B Jul 18 10:55 ./
drwxr-xr-x  5 olgabot  staff   170B Jul 18 10:55 ../
-rw-r--r--  1 olgabot  staff    74B Jul 18 10:55 config.json
-rw-r--r--  1 olgabot  staff   837B Jul 18 10:55 samples.csv


In [21]:
! head samples.csv

id,read1,read2,id,libprep,output
Undetermined_S0,s3://czbiohub-seqbot/fastqs/180628_A00111_0168_AHFVJVDMXX/rawdata/Undetermined_S0_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180628_A00111_0168_AHFVJVDMXX/rawdata/Undetermined_S0_R2_001.fastq.gz,Undetermined_S0,nextera,s3://tick-genome/dna/2018-06-28/adapter_trimmed
tick_1_S1,s3://czbiohub-seqbot/fastqs/180628_A00111_0168_AHFVJVDMXX/rawdata/tick_1_S1_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180628_A00111_0168_AHFVJVDMXX/rawdata/tick_1_S1_R2_001.fastq.gz,tick_1_S1,nextera,s3://tick-genome/dna/2018-06-28/adapter_trimmed
tick_2_S2,s3://czbiohub-seqbot/fastqs/180628_A00111_0168_AHFVJVDMXX/rawdata/tick_2_S2_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180628_A00111_0168_AHFVJVDMXX/rawdata/tick_2_S2_R2_001.fastq.gz,tick_2_S2,nextera,s3://tick-genome/dna/2018-06-28/adapter_trimmed
